# Import library and function and DF

In [ ]:
pip install lazypredict

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from lazypredict.Supervised import LazyRegressor
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.compose import make_column_selector as selector
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor

In [ ]:
import pandas as pd
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

# Get the filename
filename = next(iter(uploaded))

# Read the CSV file into a DataFrame
house_data_df = pd.read_csv(filename)

Saving housing_iteration_6_regression.csv to housing_iteration_6_regression (2).csv


# Data Cleaning

In [ ]:
house_data_df= house_data_df.drop_duplicates()

In [ ]:
house_df = house_data_df.copy()

In [ ]:
house_df = house_df.rename(columns=lambda x: x.strip())

In [ ]:
pd.set_option('display.max_colwidth', 1000)

In [ ]:
#columns_to_drop = ["Id","Alley", "MasVnrType", "FireplaceQu", "PoolQC", "Fence", "MiscFeature"]
#columns_to_drop = ["Id"]
#house_df = house_df.drop(columns=columns_to_drop)


In [ ]:
house_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [ ]:
house_df["SalePrice"]

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [ ]:
house_df.sample(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
576,577,50,RL,52.00,6292,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2009,WD,Normal,145000
742,743,20,RL,65.00,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,NaN,0,12,2007,WD,Normal,179000
156,157,20,RL,60.00,7200,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,109500
44,45,20,RL,70.00,7945,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2006,WD,Normal,141000
1284,1285,50,RL,50.00,9638,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,3,2010,WD,Normal,169000
914,915,160,FV,30.00,3000,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2009,New,Partial,173733
520,521,190,RL,60.00,10800,Pave,Grvl,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2008,WD,Normal,106250
1099,1100,20,RL,82.00,11880,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2009,COD,Abnorml,157000
990,991,60,RL,82.00,9452,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,348000
290,291,60,RL,120.00,15611,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2006,New,Partial,233230


In [ ]:
house_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [ ]:
house_data_df.isna().sum().sample(10)

LotFrontage      259
ExterCond          0
SaleType           0
Electrical         1
BsmtHalfBath       0
GarageCars         0
LandSlope          0
BsmtQual          37
FireplaceQu      690
Alley           1369
dtype: int64

# Split the data

In [ ]:
# Create our feature vector
X = house_df.drop(columns=["Id"]).copy()

# Target feature
y = X.pop("SalePrice")

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Splitting categorical & numerical



In [ ]:
house_df["SalePrice"]

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

# Get the list of ordinal and non ordinal from the categorical colum X_cat

In [ ]:
X_num = X_train.select_dtypes(include="number").copy()
X_cat = X_train.select_dtypes(exclude="number").copy()

# List of ordinal columns
ordinal_cols_names = ["LotShape", "Utilities", "HouseStyle", "OverallQual",
                      "OverallCond", "ExterQual", "ExterCond", "BsmtQual",
                      "BsmtCond", "GarageType", "PoolQC", "Fence",
                      "SaleCondition"]

ordinal_cols = X_cat.columns.get_indexer(ordinal_cols_names)

# List of non_ordinal columns
non_ordinal_cols_names = [col for col in X_cat.columns if col not in ordinal_cols_names]
onehot_cols = X_cat.columns.get_indexer(non_ordinal_cols_names)

# Define the ranking for each ordinal column
ordinal_rankings = [
    ["IR3", "IR2", "IR1", "Reg"],  # LotShape
    ["ELO", "NoSeWa", "NoSewr", "AllPub"],  # Utilities
    ["1Story", "1.5Fin", "1.5Unf", "2Story", "2.5Fin", "2.5Unf", "SFoyer", "SLvl"],  # HouseStyle
    ["Very Poor", "Poor", "Fair", "Below Average", "Average", "Above Average", "Good", "Very Good", "Excellent", "Very Excellent"],  # OverallQual
    ["Very Poor", "Poor", "Fair", "Below Average", "Average", "Above Average", "Good", "Very Good", "Excellent", "Very Excellent"],  # OverallCond
    ["Po", "Fa", "TA", "Gd", "Ex"],  # ExterQual
    ["Po", "Fa", "TA", "Gd", "Ex"],  # ExterCond
    ["NA", "Po", "Fa", "TA", "Gd", "Ex"],  # BsmtQual
    ["NA", "Po", "Fa", "TA", "Gd", "Ex"],  # BsmtCond
    ['Attchd', 'BuiltIn', 'Detchd', 'Basment', 'CarPort', '2Types'],  # GarageType
    ['NA', 'Fa', 'TA', 'Gd', 'Ex'],  # PoolQC
    ['NA', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv'],  # Fence
    ["Normal", "Abnorml", "AdjLand", "Alloca", "Family", "Partial"]  # SaleCondition
]

# Define the categorical encoder
categorical_preprocessor = make_column_transformer(
    (OrdinalEncoder(categories=ordinal_rankings,
                    encoded_missing_value=-1,
                    handle_unknown='use_encoded_value',
                    unknown_value=-1), ordinal_cols),
    (OneHotEncoder(sparse=False, handle_unknown="ignore"), onehot_cols)
)

# Create the categorical pipeline
categorical_pipe = make_pipeline(SimpleImputer(strategy="constant", fill_value="N_A"),
                                 categorical_preprocessor)

# Create the numeric pipeline
numeric_pipe = make_pipeline(SimpleImputer(strategy='mean'))

# Create the preprocessor column transformer
preprocessor = make_column_transformer(
    (numeric_pipe, X_num.columns),
    (categorical_pipe, X_cat.columns)
)


In [ ]:
categorical_pipe

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='N_A', strategy='constant')),
                ('columntransformer',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Grvl',
                                                                              'Pave'],
                                                                             ['NA',
                                                                              'Grvl',
                                                                              'Pave'],
                                                                             ['IR3',
                                                                              'IR2',
                                                                              'IR1',
                                                                              'Reg'],
                                                                             ['ELO',
                                                                              'NoSeWa',
                                                                              'NoSewr',
                                                                              'AllPub'],
                                                                             ['1Story',
                                                                              '1.5Fin',
                                                                              '1.5Unf',
                                                                              '2Story',
                                                                              '2.5Fin',
                                                                              '2.5Unf',
                                                                              'SFoyer',
                                                                              'SLv...
                                                                             ['Normal',
                                                                              'Abnorml',
                                                                              'AdjLand',
                                                                              'Alloca',
                                                                              'Family',
                                                                              'Partial']],
                                                                 encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  array([ 1,  2,  3,  5, 12, -1, -1, 18, 19, 21, -1, 33, 38, 39, 42])),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  array([ 0,  4,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 20, 22, 23, 24,
       25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 40, 41]))]))])

In [ ]:
# create the numerique pipeline
#numeric_pipe = make_pipeline(SimpleImputer(strategy='mean'))

In [ ]:
# preprocessor = make_column_transformer(
 #   (numeric_pipe, X_num.columns),
#    (categorical_pipe, X_cat.columns)
#)


In [ ]:
X_train_processed = preprocessor.fit_transform(X_train)

X_train_processed

array([[2.000e+01, 7.000e+01, 8.400e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [6.000e+01, 5.900e+01, 7.837e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [3.000e+01, 6.700e+01, 8.777e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       ...,
       [2.000e+01, 6.000e+01, 8.172e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [5.000e+01, 5.500e+01, 7.642e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [1.200e+02, 5.300e+01, 3.684e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00]])

In [ ]:
# 2.Manually apply MinMaxScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_processed)
X_train_scaled


array([[0.        , 0.16780822, 0.0331861 , ..., 0.        , 0.        ,
        1.        ],
       [0.23529412, 0.13013699, 0.03055458, ..., 0.        , 0.        ,
        1.        ],
       [0.05882353, 0.15753425, 0.03494823, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.        , 0.13356164, 0.0321204 , ..., 0.        , 0.        ,
        1.        ],
       [0.17647059, 0.11643836, 0.02964313, ..., 0.        , 0.        ,
        1.        ],
       [0.58823529, 0.10958904, 0.01114305, ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
X_train_scaled.shape

In [ ]:
# 4. Apply SelectKBest
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

selector = SelectKBest(score_func=mutual_info_regression, k=10)  # Choose k as desired
X_train_kbest = selector.fit_transform(X_train_scaled, y_train)

# Get selected feature indices from SelectKBest
selected_indices = selector.get_support(indices=True)

# Translate selected indices to feature names
feature_names_processed= preprocessor.get_feature_names_out()
selected_feature_names = [feature_names_processed[i] for i in selected_indices]

print("Top features selected by SelectKBest:", selected_feature_names)

X_train_kbest

Top features selected by SelectKBest: ['pipeline-1__MSSubClass', 'pipeline-1__OverallQual', 'pipeline-1__YearBuilt', 'pipeline-1__TotalBsmtSF', 'pipeline-1__1stFlrSF', 'pipeline-1__GrLivArea', 'pipeline-1__GarageCars', 'pipeline-1__GarageArea', 'pipeline-2__ordinalencoder__ExterQual', 'pipeline-2__ordinalencoder__BsmtQual']


array([[0.        , 0.44444444, 0.61594203, ..., 0.20733427, 0.33333333,
        0.66666667],
       [0.23529412, 0.55555556, 0.87681159, ..., 0.26798307, 0.66666667,
        0.83333333],
       [0.05882353, 0.44444444, 0.27536232, ..., 0.        , 0.33333333,
        0.5       ],
       ...,
       [0.        , 0.44444444, 0.60144928, ..., 0.40338505, 0.33333333,
        0.66666667],
       [0.17647059, 0.66666667, 0.33333333, ..., 0.15232722, 0.66666667,
        0.66666667],
       [0.58823529, 0.66666667, 0.97826087, ..., 0.46544429, 0.66666667,
        0.83333333]])

In [ ]:
from sklearn.feature_selection import f_regression
# 4. Apply SelectKBest
selector = SelectKBest(score_func=f_regression, k=10)  # Choose k as desired
X_train_kbest = selector.fit_transform(X_train_scaled, y_train)

# Get selected feature indices from SelectKBest
selected_indices = selector.get_support(indices=True)

# Translate selected indices to feature names
selected_feature_names = [feature_names_processed[i] for i in selected_indices]

print("Top features selected by SelectKBest:", selected_feature_names)

X_train_kbest

Top features selected by SelectKBest: ['pipeline-1__OverallQual', 'pipeline-1__TotalBsmtSF', 'pipeline-1__1stFlrSF', 'pipeline-1__GrLivArea', 'pipeline-1__FullBath', 'pipeline-1__TotRmsAbvGrd', 'pipeline-1__GarageCars', 'pipeline-1__GarageArea', 'pipeline-2__ordinalencoder__ExterQual', 'pipeline-2__ordinalencoder__BsmtQual']


array([[0.44444444, 0.21505728, 0.2248738 , ..., 0.20733427, 0.33333333,
        0.66666667],
       [0.55555556, 0.13076923, 0.10670032, ..., 0.26798307, 0.66666667,
        0.83333333],
       [0.44444444, 0.13027823, 0.10601193, ..., 0.        , 0.33333333,
        0.5       ],
       ...,
       [0.44444444, 0.14140753, 0.12161542, ..., 0.40338505, 0.33333333,
        0.66666667],
       [0.66666667, 0.1492635 , 0.13262965, ..., 0.15232722, 0.66666667,
        0.66666667],
       [0.66666667, 0.22471358, 0.28017439, ..., 0.46544429, 0.66666667,
        0.83333333]])

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

pipeline = make_pipeline(
    preprocessor,
    MinMaxScaler(),
    SelectKBest(score_func=mutual_info_regression),  # or score_func=f_regression
    RandomForestRegressor()
)

# Define the hyperparameter grid for RandomizedSearchCV
param_grid = {
    "columntransformer__pipeline-1__simpleimputer__strategy": ["mean", "median"],  # Use the correct name for numeric pipeline
    "randomforestregressor__n_estimators": range(50, 400, 50),
    "randomforestregressor__max_depth": range(1, 20, 2),
    "randomforestregressor__min_samples_split": range(2, 20, 2),
    "selectkbest__k": range(1, 010, 2)
}

# Perform RandomizedSearchCV for hyperparameter tuning
search = RandomizedSearchCV(pipeline,
                            param_distributions=param_grid,
                            n_iter=30,
                            cv=5,
                            scoring="neg_mean_squared_error",  # Use 'neg_mean_squared_error'
                            verbose=1,
                            n_jobs=-1)

# Fit the model to the training data
search.fit(X_train, y_train)

# Create a dictionary to store the average scores
avg_scores = {"rforest": search.best_score_}

avg_scores


Fitting 5 folds for each of 30 candidates, totalling 150 fits


{'rforest': -931748178.4797695}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import GradientBoostingRegressor  # Import the GradientBoostingRegressor

pipeline = make_pipeline(
    preprocessor,
    MinMaxScaler(),
    SelectKBest(score_func=mutual_info_regression),  # or score_func=f_regression
    GradientBoostingRegressor()  # Change RandomForestRegressor to GradientBoostingRegressor
)

# Define the hyperparameter grid for RandomizedSearchCV
param_grid = {
    "columntransformer__pipeline-1__simpleimputer__strategy": ["mean", "median"],
    "gradientboostingregressor__n_estimators": range(50, 400, 50),  # Change the parameter names for GradientBoostingRegressor
    "gradientboostingregressor__max_depth": range(1, 20, 2),  # Change the parameter names for GradientBoostingRegressor
    "gradientboostingregressor__min_samples_split": range(2, 20, 2),  # Change the parameter names for GradientBoostingRegressor
    "selectkbest__k": range(1, 30, 2)  # Make sure to adjust the maximum value for selectkbest__k as needed
}

# Perform RandomizedSearchCV for hyperparameter tuning
search = RandomizedSearchCV(pipeline,
                            param_distributions=param_grid,
                            n_iter=30,
                            cv=5,
                            scoring="neg_mean_squared_error",
                            verbose=1,
                            n_jobs=-1)

# Fit the model to the training data
search.fit(X_train, y_train)

# Create a dictionary to store the average scores
avg_scores = {"gradient_boosting": search.best_score_}  # Change the model name

avg_scores


Fitting 5 folds for each of 30 candidates, totalling 150 fits


{'gradient_boosting': -881159217.4074367}

# decision tree regressor

In [ ]:
X_train_preprocessor = preprocessor.fit_transform(X_train)

In [ ]:
X_train_preprocessor

array([[2.000e+01, 7.000e+01, 8.400e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [6.000e+01, 5.900e+01, 7.837e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [3.000e+01, 6.700e+01, 8.777e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       ...,
       [2.000e+01, 6.000e+01, 8.172e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [5.000e+01, 5.500e+01, 7.642e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [1.200e+02, 5.300e+01, 3.684e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00]])

In [ ]:
dt_pipeline = make_pipeline(preprocessor,
                            StandardScaler(),
                            DecisionTreeRegressor())

dt_pipeline.fit(X_train, y_train)

#dt_predictions = dt_pipeline.predict(X_test)

In [ ]:
X_test.info()

In [ ]:
dt_predictions = dt_pipeline.predict(X_test)

In [ ]:
dt_predictions

In [ ]:
from sklearn.metrics import r2_score

dt_r2 = r2_score(y_true = y_test,
                 y_pred = dt_predictions)



In [ ]:
y_test

In [ ]:
dt_r2

0.7636962032827042

# SGDRegressor

In [ ]:
sgd_pipeline = make_pipeline(preprocessor,
                            StandardScaler(),
                            SGDRegressor())

sgd_pipeline.fit(X_train, y_train)

sgd_predictions = sgd_pipeline.predict(X_test)

In [ ]:
sgd_r2 = r2_score(y_true = y_test,
                 y_pred = sgd_predictions)

In [ ]:
sgd_r2

-46.58773581068521

#LinearRegression

In [ ]:
lr_pipeline = make_pipeline(preprocessor,
                            StandardScaler(),
                            LinearRegression())

lr_pipeline.fit(X_train, y_train)

lr_predictions = lr_pipeline.predict(X_test)

In [ ]:
lr_r2 = r2_score(y_true = y_test,
                 y_pred = lr_predictions)

In [ ]:
lr_r2

-3.904400311902329e+22